Source Extraction with DAOSTarFinder 

Fair amount of code from: https://photutils.readthedocs.io/en/stable/detection.html

In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture, EllipticalAperture

import pandas as pd 

In [3]:
sources = pd.read_csv("23456_sources.csv")

sources

,Unnamed: 0,id,xcentroid,ycentroid,sharpness,roundness1,roundness2,npix,sky,peak,flux,mag
0,0,1,1395.580099,36.247815,0.569854,-0.405312,0.607445,49,0.0,0.013523,1.770291,-0.620112
1,1,2,1651.213208,41.241046,0.570932,0.616550,-0.235443,49,0.0,0.011518,1.570322,-0.489972
2,2,3,1631.803127,56.205071,0.653343,-0.223447,0.466133,49,0.0,0.012183,1.504983,-0.443829
3,3,4,1659.953970,55.195978,0.693991,-0.342607,-0.705724,49,0.0,0.011493,1.426781,-0.385893
4,4,5,1630.202409,63.900624,0.821820,-0.341085,-0.112867,49,0.0,0.010775,1.049387,-0.052340
...,...,...,...,...,...,...,...,...,...,...,...,...
2503,2503,2504,1332.194913,3012.979123,0.520923,-0.316986,-0.591994,49,0.0,0.007527,1.417007,-0.378430
2504,2504,2505,1493.701853,3013.292060,0.308525,-0.741443,0.795482,49,0.0,0.014486,1.522278,-0.456235
2505,2505,2506,1509.289430,3020.618508,0.561032,-0.552576,-0.576782,49,0.0,0.016001,2.136026,-0.824016
2506,2506,2507,1602.758315,3020.779658,0.283894,-0.196238,-0.328335,49,0.0,0.004442,1.110665,-0.113958


In [ ]:
# adding new column to sources that uses roundedness2 (R2) to calculate the ratio between hx and hy, the height of the 
# best fitting gaussian to the marginal x or y axis distribution of the unconvulved source data. 

# original roundedness2 equation: 2*(hx-hy/(hx+hy) = R2
# ratio calculation derived from above: hx/hy = (2+R2)/(2-R2)

# for example, where hx and hy are the same (hx/hy=1), the source is a perfect circle and R2 would be 0
# where R2 is negative, hy is bigger than hx & where R2 is positive, hx is bigger than hy 

sources["hx_hy_ratio"] = (2 + sources["roundness2"]) / (2 - sources["roundness2"])


In [10]:
# converting hx and hy to distances in the x and y direction based on if the ratio (hx_hy_ratio) is less than or 
# greater than 1. 

# b is base size in pixels (or, what minimum size would you want your circle to be if b was the radius)

# x is the distance in the x direction, y is the distance in the y direction. depending on which is greater, x and 
# y will determine the semi major and minor axes of the ellipse. 

# if hx_hy_ratio > 1 then x = b * hx_hy_ratio and y = b (so x is hx_hy_ratio bigger than y)
# if hx_hy_ratio < 1 then x = b and y = b * (hx_hy_ratio)**-1

# function below to calculate x and y 

def calc_x(ratio, base_size): 
    if ratio > 1: 
        x = base_size * ratio 
    elif ratio <= 1: 
        x = base_size 
    return x

def calc_y(ratio, base_size): 
    if ratio < 1: 
        y = base_size * ratio**-1
    elif ratio >= 1: 
        y = base_size 
    return y


In [ ]:
base_size = 6 # min size in pixels of output when a circle (aka the semiminor axis)

sources['x_len'] = sources.apply(lambda sources: calc_x(sources['hx_hy_ratio'], base_size), axis=1)
sources['y_len'] = sources.apply(lambda sources: calc_y(sources['hx_hy_ratio'], base_size), axis=1)
sources['theta'] = sources['hx_hy_ratio'].apply(lambda x: 0 if x <= 1  else (np.pi/2))

In [ ]:
# Plotting using elliptical aperture from photo utils is a little annoying because it only takes the semi major and
# minor axes as floats, not arrays. so to plot, we have to make a new EllipticalAperture object for each source 

# this idea ^ doesn't seem very efficient 

positions = np.transpose((sources['xcentroid'], sources['ycentroid']))

# masking cnt image
masked_cnt = zero_flag_and_edge(cnt, flag, edge)

In [ ]:
sources.loc[0,'x_len']

In [ ]:
# plotting 

plt.imshow(centile_clip(masked_cnt,centiles=(0,99.9)),interpolation='none')
for i in range(len(sources)): 
    aperture = EllipticalAperture(positions[i], 
                a=max(sources.loc[i,'x_len'],sources.loc[i,'y_len']), 
                b=min(sources.loc[i,'x_len'],sources.loc[i,'y_len']),
                theta = sources.loc[i,'theta'])
    plt.show(aperture.plot(color='red', lw=sources.loc[i,'sharpness']*2, alpha=0.5))

In [5]:
# comparing the results of using ratio & theta (theta = change x axis orientation of kernel in deg & ratio makes
# kernel more elliptical, ratio = 1 would be a circle). 

# radius 0.8, theta 45 deg
source_r8_th45 = pd.read_csv("23456_sources_45x.csv")

# radius 0.8, theta 0 (normal)
source_r8_th0 = pd.read_csv("23456_sources_0x.csv")

# radius 0 (circle), theta 45 
source_th45 = pd.read_csv("23456_sources_45th.csv")


In [8]:
sources

,Unnamed: 0,id,xcentroid,ycentroid,sharpness,roundness1,roundness2,npix,sky,peak,flux,mag
0,0,1,1395.580099,36.247815,0.569854,-0.405312,0.607445,49,0.0,0.013523,1.770291,-0.620112
1,1,2,1651.213208,41.241046,0.570932,0.616550,-0.235443,49,0.0,0.011518,1.570322,-0.489972
2,2,3,1631.803127,56.205071,0.653343,-0.223447,0.466133,49,0.0,0.012183,1.504983,-0.443829
3,3,4,1659.953970,55.195978,0.693991,-0.342607,-0.705724,49,0.0,0.011493,1.426781,-0.385893
4,4,5,1630.202409,63.900624,0.821820,-0.341085,-0.112867,49,0.0,0.010775,1.049387,-0.052340
...,...,...,...,...,...,...,...,...,...,...,...,...
2503,2503,2504,1332.194913,3012.979123,0.520923,-0.316986,-0.591994,49,0.0,0.007527,1.417007,-0.378430
2504,2504,2505,1493.701853,3013.292060,0.308525,-0.741443,0.795482,49,0.0,0.014486,1.522278,-0.456235
2505,2505,2506,1509.289430,3020.618508,0.561032,-0.552576,-0.576782,49,0.0,0.016001,2.136026,-0.824016
2506,2506,2507,1602.758315,3020.779658,0.283894,-0.196238,-0.328335,49,0.0,0.004442,1.110665,-0.113958


In [6]:
source_th45

,Unnamed: 0,id,xcentroid,ycentroid,sharpness,roundness1,roundness2,npix,sky,peak,flux,mag
0,0,1,1395.580099,36.247815,0.569854,-0.405312,0.607445,49,0.0,0.013523,1.770291,-0.620112
1,1,2,1651.213208,41.241046,0.570932,0.616550,-0.235443,49,0.0,0.011518,1.570322,-0.489972
2,2,3,1631.803127,56.205071,0.653343,-0.223447,0.466133,49,0.0,0.012183,1.504983,-0.443829
3,3,4,1659.953970,55.195978,0.693991,-0.342607,-0.705724,49,0.0,0.011493,1.426781,-0.385893
4,4,5,1630.202409,63.900624,0.821820,-0.341085,-0.112867,49,0.0,0.010775,1.049387,-0.052340
...,...,...,...,...,...,...,...,...,...,...,...,...
2503,2503,2504,1332.194913,3012.979123,0.520923,-0.316986,-0.591994,49,0.0,0.007527,1.417007,-0.378430
2504,2504,2505,1493.701853,3013.292060,0.308525,-0.741443,0.795482,49,0.0,0.014486,1.522278,-0.456235
2505,2505,2506,1509.289430,3020.618508,0.561032,-0.552576,-0.576782,49,0.0,0.016001,2.136026,-0.824016
2506,2506,2507,1602.758315,3020.779658,0.283894,-0.196238,-0.328335,49,0.0,0.004442,1.110665,-0.113958


In [ ]:
# vals for plotting comparison 

source_r8_th45["hx_hy_ratio"] = (2 + source_r8_th45["roundness2"]) / (2 - source_r8_th45["roundness2"])

source_r8_th45['x_len'] = source_r8_th45.apply(lambda source_r8_th45: calc_x(source_r8_th45['hx_hy_ratio'], base_size), axis=1)
source_r8_th45['y_len'] = source_r8_th45.apply(lambda source_r8_th45: calc_y(source_r8_th45['hx_hy_ratio'], base_size), axis=1)
source_r8_th45['theta'] = source_r8_th45['hx_hy_ratio'].apply(lambda x: 0 if x <= 1  else (np.pi/2))


positions_r8_th45 = np.transpose((source_r8_th45['xcentroid'], source_r8_th45['ycentroid']))


source_r8_th0["hx_hy_ratio"] = (2 + source_r8_th0["roundness2"]) / (2 - source_r8_th0["roundness2"])

source_r8_th0['x_len'] = source_r8_th0.apply(lambda source_r8_th0: calc_x(source_r8_th0['hx_hy_ratio'], base_size), axis=1)
source_r8_th0['y_len'] = source_r8_th0.apply(lambda source_r8_th0: calc_y(source_r8_th0['hx_hy_ratio'], base_size), axis=1)
source_r8_th0['theta'] = source_r8_th0['hx_hy_ratio'].apply(lambda x: 0 if x <= 1  else (np.pi/2))


positions_r8_th0 = np.transpose((source_r8_th0['xcentroid'], source_r8_th0['ycentroid']))

In [ ]:
# plotting 

plt.imshow(centile_clip(masked_cnt,centiles=(0,99.9)),interpolation='none')
"""
for i in range(len(sources)): 
    aperture = EllipticalAperture(positions[i], 
                a=max(sources.loc[i,'x_len'],sources.loc[i,'y_len']), 
                b=min(sources.loc[i,'x_len'],sources.loc[i,'y_len']),
                theta = sources.loc[i,'theta'])
    plt.show(aperture.plot(color='red', lw=sources.loc[i,'sharpness']*2, alpha=0.5)) 
"""
    
for i in range(len(source_r8_th0)): 
    aperture = EllipticalAperture(positions_r8_th0[i], 
                a=max(source_r8_th0.loc[i,'x_len'],source_r8_th0.loc[i,'y_len']), 
                b=min(source_r8_th0.loc[i,'x_len'],source_r8_th0.loc[i,'y_len']),
                theta = source_r8_th0.loc[i,'theta'])
    plt.show(aperture.plot(color='white', lw=source_r8_th0.loc[i,'sharpness']*2, alpha=1))
    
 
for i in range(len(source_r8_th45)): 
    aperture = EllipticalAperture(positions_r8_th45[i], 
                a=max(source_r8_th45.loc[i,'x_len'],source_r8_th45.loc[i,'y_len']), 
                b=min(source_r8_th45.loc[i,'x_len'],source_r8_th45.loc[i,'y_len']),
                theta = source_r8_th45.loc[i,'theta'] + 0.785398)
    plt.show(aperture.plot(color='fuchsia', lw=source_r8_th45.loc[i,'sharpness']*2, alpha=1))

In [9]:
def centile_clip(image, centiles=(1, 99)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    if isinstance(image, np.ma.MaskedArray):
        return result
    return result.data

def plot_centiles(array, upper_bounds):
    figures, axes = [], []
    for upper_bound in (upper_bounds):
        fig, ax = plt.subplots()
        ax.imshow(centile_clip(cnt, centiles=(0, upper_bound)))
        ax.set_title(str(upper_bound))
        figures.append(fig), axes.append(ax)
    return figures, axes


In [4]:
# for images too (also will use mpl)
import fitsio
from astropy.io import fits
import sys 
import numpy.ma as ma
from gPhoton.reference import eclipse_to_paths
from gPhoton.coadd import zero_flag_and_edge
import matplotlib as mpl
import matplotlib.pyplot as plt 

In [11]:
%time hdul = fitsio.FITS(galex_file_paths[band]['image'])
%time cnt, flag, edge = [hdu.read() for hdu in hdul[1:4]]

NameError: name 'galex_file_paths' is not defined

NameError: name 'hdul' is not defined

In [ ]:
# info 
eclipse, band, depth, compression = 23456, "NUV", 30, "rice"

# reading in images 
galex_file_paths = eclipse_to_paths(
    eclipse, '/home/bekah/gphoton_working/test_data', depth, compression
)

#galex_file_paths

In [ ]:
# selects cnt a diff way 

#hdul = fits.open(galex_file_paths[band]['movie'])
#cnt = hdul[1].data

In [ ]:
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=6.)
norm = ImageNormalize(stretch=SqrtStretch())

In [ ]:
masked_cnt = zero_flag_and_edge(cnt, flag, edge)

In [ ]:
plt.imshow(centile_clip(masked_cnt,centiles=(0,99.9)),interpolation='none')
plt.show(apertures.plot(color='red', lw=1.5, alpha=0.5))

In [ ]:
from astropy.wcs import WCS
from gPhoton.coords.wcs import sky_box_to_image_box

wcs_obj = WCS(hdul[1].read_header())
wcs_obj

In [ ]:
box_corners = sky_box_to_image_box(((324.1, -2.3), (324.2, -2.4)), wcs_obj)
plt.imshow(masked_cnt[box_corners[0]:box_corners[1], box_corners[2]:box_corners[3]])
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.show